# crawl all waterschap beschikkingen

* marx
* 2023-10-18
* variant op Den Haag sjabloon <localhost:8888/notebooks/WooIR/wobir/Projects/WooQualityCheck/Gemeentes/DenHaag/Crawl_gm0518_2i.ipynb>

## Sjaboon  (wij doen het hier net effe anders)

[Den Haag s Woo dossiers](https://woo.denhaag.nl/wob-besluiten/) zijn een mooi voorbeeld van een publicatie-methode die heel goed netjes te scrape  valt. Het bestaat uit 3 onderdelen:

1. Een pagina met een lijst van alle woo-dossiers,  met een link naar een aparte pagina voor elk dossier.
    * Het gebruik van "pagination in de URL" om naar de volgende pagina in die lijst te gaan.
    * **Stap 1 i het extractiescript gebruikt dit.**
2. Elk Woo-dossier heeft een eigen vaste pagina met een eigen URL.
    * Die worden in **stap 2** verzameld inn een lijst.
3. Elke pagina met een Woo-dossier heeft een vaste structuur die via XPad om te zettenn is naar ons schema.
    * Meta data op de HTML pagina met een vast schema (dc, og  of iets vergelijkbaars) maakt het verzamelen van metadata op dossier  niveau vaak eenvoudiger en robuuster.
    * De extractioe per dossier gebeurt in **stap 3**.
4. Tenslotte doenn we de extractie over alle urls van xde woo-dossiers, en slaan dat allemaal op in eeen json datafile. **stap 4 en 5**.

In [5]:

import re
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import json
import requests
import gzip

# globals

In [48]:
DC_TYPE = '2k'
DC_PUBLISHER='ws0646'
DC_CREATOR='Waterschap Hunze En Aa\'s'

# Extractie functies

In [28]:
def crawl_all_pages(url,page_range=range(100)):
    '''Crawl all pages from a paginated list page with URL and pagerange.
    Simply concatenate all of them and return that as a strng. 
    From that string we will extract all pages we want'''
    pages=''
    for p in page_range:
        u= url+str(p)
        #print (u)
        ef = !curl -s "$u"
        pages=pages+'\n'+ ''.join(ef)
    return pages




def xpadje(element,path_expression):
    if element.xpath(path_expression):
        return element.xpath(path_expression)[0]
    else:
        
        return ''
    
def pak_meta(element,property_value='dc:title'):
    '''Geef de waarde van het @content attribuut in het meta element met @property waarde property_value.
    Als het niet bestaat geef lege string'''
    xpad= './/meta[@property="WAARDE"]/@content'.replace('WAARDE',property_value)
    return xpadje(element, xpad)
    
  
    
def extract_links(woodata,dc_source_xpad='@href', dc_title_xpad='.//text()'):
    '''Extract the list of document dicts from woodata which is a list of a elements from the HTML page.
    One usually only has to finetune the dc_title_xpad'''
    L=[]
    for e in woodata:
        D=dict()
        
        D["dc_source"]= xpadje(e,dc_source_xpad)
        D["dc_title"]= ''.join(e.xpath(dc_title_xpad)).strip() 
        if 'besluit'.lower() in D["dc_title"].lower() :
            D["dc_type"]='besluit' 
        elif 'verzoek'.lower() in D["dc_title"].lower() :
            D["dc_type"]='verzoek' 
        elif 'inventaris'.lower() in D["dc_title"].lower() :
            D["dc_type"]='inventarislijst' 
        else:
            D["dc_type"]='bijlage' 
        D["foi_url_on_web"]= True #'true'
        L.append(D) 
    return  L

# step 1 crawl all pages with links to woo dossiers

* we halen hem op in stapjes van 10 steeds

## todo

* Die 600 is hardcoded. Niet goed. Ga eerst kijken hoeveel hits er zijn en pas script dan dynamisch aan.

In [42]:
URL=('https://zoek.officielebekendmakingen.nl/resultaten?'
     'q=(c.product-area==%22officielepublicaties%22)and((w.organisatietype==%22waterschap%22))'
     'and(((w.publicatienaam==%22Tractatenblad%22))or((w.publicatienaam==%22Staatsblad%22))'
     'or((w.publicatienaam==%22Staatscourant%22))or((w.publicatienaam==%22Gemeenteblad%22))'
     'or((w.publicatienaam==%22Provinciaal%20blad%22))or((w.publicatienaam==%22Waterschapsblad%22))'
     'or((w.publicatienaam==%22Blad%20gemeenschappelijke%20regeling%22)))'
     '%20AND%20dt.creator==%22Waterschap%20Hunze%20en%20Aa%27s%22&'
     'zv=&col=AlleBekendmakingen&svel=Publicatiedatum&svol=Aflopend&sf=po|Waterschap+Hunze+en+Aa%27s&pg='
    )
page_range=range(10,600,10) ## todo dit moet nog uitgeprogrameeerd worden
pages = crawl_all_pages(URL,page_range )

# step 2 extract links to woo dosiers

* fine tune 

In [46]:
woos=re.findall(r'href="(.*?)"',pages)

woos= [u for u in woos if u.endswith('html') and not '/' in u ]
woos=[u.replace('.html','') for u in woos]
print(len(woos), len(set(woos)), woos[:5])
woos=list(set(woos))

34952 528 ['wsb-2023-12405', 'wsb-2023-12405', 'wsb-2023-12217', 'wsb-2023-12217', 'wsb-2023-12219']


# step 3 download xml files

In [50]:
def make_urls(ob_id):
    u= 'https://zoek.officielebekendmakingen.nl/'+ ob_id  
    return (u+'.xml', u+'/metadata.xml') 

In [49]:
#!mkdir $DC_PUBLISHER

# stap 4

* Extraheer u de data voor elk woodossier i n `woos`.

In [ ]:
#%%time

%cd $DC_PUBLISHER
for ob_id in woos[:2]:
    try:
        data,metadata= make_urls(ob_id)
        #!cd $DC_PUBLISHER && { curl -O $data ;curl -O $metadata; cd -; }
        metafilename= ob_id+".metadata.xml"
        !curl  --silent -O $data ;curl --silent  $metadata >$metafilename;
    except:
        print('WRONG',ob_id)
# gzip all files
!gzip *
%cd ..
!ls  $DC_PUBLISHER |wc
!du -hs $DC_PUBLISHER

# step 4 trasform into pandas   

For doig data-analysis and transformation

In [71]:
df= pd.read_xml('ws0646/wsb-2023-12405.metadata.xml.gz',xpath=".//metadata")
df

,name,scheme,content,metadata
0,OVERHEID.organisationType,OVERHEID.Organisatietype,waterschap,NaN
1,OVERHEIDop.publicationName,NaN,Waterschapsblad,NaN
2,OVERHEIDop.doctype,NaN,"Officiële Publicaties, versie 1.1",NaN
3,OVERHEIDop.configuratie,NaN,https://repository.officiele-overheidspublicat...,NaN
4,DC.creator,OVERHEID.Waterschap,Waterschap Hunze en Aa's,NaN
5,DC.type,OVERHEIDop.Rubriek,omgevingsvergunning,NaN
6,DC.type,OVERHEID.Informatietype,officiële publicatie,NaN
7,DCTERMS.publisher,OVERHEID.Waterschap,Waterschap Hunze en Aa's,NaN
8,OVERHEID.authority,OVERHEID.Waterschap,Waterschap Hunze en Aa's,NaN
9,OVERHEID.category,OVERHEID.TaxonomieBeleidsagendaDecentraal,Ruimte en infrastructuur | Organisatie en beleid,NaN


# step 5 trasform into json 

for crawler, and store in woogle

In [8]:
jsonfilename=DC_PUBLISHER+'json.gz'
with gzip.open(jsonfilename, 'w') as fout:
    fout.write(json.dumps(D).encode('utf-8'))  
#!ls -lh $jsonfilename

In [9]:
# if you want to open the file again
#with gzip.open(jsonfilename, 'r') as fin:
#    data = json.loads(fin.read().decode('utf-8'))
#len(data),data[:2]